In [ ]:
# %cd Yet-Another-EfficientDet-Pytorch

In [ ]:
%cd ..

In [ ]:
import argparse
import datetime
import os
import traceback

import numpy as np
import torch
import yaml

import albumentations as A
from tensorboardX import SummaryWriter
from torch import nn
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm.autonotebook import tqdm

from efficientdet.mydataset import CocoDataset, CocoDatasetForAlbumentations
from efficientdet.mydataset import Resizer, Normalizer, Augmenter, collater
from efficientdet.loss import FocalLoss
from utils.sync_batchnorm import patch_replication_callback
from utils.utils import replace_w_sync_bn, CustomDataParallel, get_last_weights, init_weights, boolean_string

import cv2

In [ ]:
class Params:
    def __init__(self, project_file):
        with open(project_file) as f:
            self.params = yaml.safe_load(f)

    def __getattr__(self, item):
        return self.params.get(item, None)

# print current path
print(os.getcwd())

params = Params(f'./projects/fashion.yml')
training_params = {
    'batch_size': 1,
    'shuffle': False,
    'drop_last': True,
    'collate_fn': collater,
    'num_workers': 1
}
# make option with dot dict
opt = {
    'compound_coef': 7,
    'num_workers': 1,
    'data_path': '../../',
    'compound_coef': 0,
}
opt = argparse.Namespace(**opt)

In [ ]:
import cv2
import matplotlib.pyplot as plt

BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White

def random_box_color():
    # random hsv color: saturation, brightness is always maximum
    hsv_color = np.array([np.random.randint(0, 179), 255, 200], dtype=np.uint8)

    # convert (h, s, v) to (r, g, b) color space
    rgb_color = cv2.cvtColor(np.array([[hsv_color]], dtype=np.uint8), cv2.COLOR_HSV2RGB)[0][0]

    return rgb_color.tolist()

print(random_box_color())


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, x_max, y_max = map(int, bbox)
   
    # print(img)
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), color, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name, scores):
    img = image.copy()
    for bbox, category_id, score in zip(bboxes, category_ids, scores):
        if score < 0.3:
            continue
        if category_id not in category_id_to_name:
            continue
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name, random_box_color())
    plt.figure(figsize=(12, 12))
    plt.axis('off')
    plt.imshow(img)

In [ ]:
input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536, 1536]

input_size = input_sizes[opt.compound_coef]


def getLoader():
    training_transform = A.Compose([
        A.Normalize(),
        # A.HorizontalFlip(p=0.5),
        A.LongestMaxSize(max_size=input_sizes[opt.compound_coef]),
        # A.BBoxSafeRandomCrop(height=input_size, width=input_size, p=1.0),
        A.PadIfNeeded(min_height=input_size, min_width=input_size, position=A.PadIfNeeded.PositionType.TOP_LEFT, border_mode=cv2.BORDER_CONSTANT, value=[0, 0, 0], p=1.0),
    ], bbox_params=A.BboxParams(format='coco', label_fields=['category_id']))

    training_set = CocoDatasetForAlbumentations(
        root_dir=os.path.join(opt.data_path, params.project_name),
        set=params.train_set,
        transform=training_transform
    )

    return DataLoader(training_set, **training_params)

def getOriginalLoader():

    training_transform = transforms.Compose([
        Normalizer(mean=params.mean, std=params.std),
        Resizer(input_sizes[opt.compound_coef])
    ])
    training_set = CocoDataset(
        root_dir=os.path.join(opt.data_path, params.project_name),
        set=params.train_set,
        transform=training_transform
    )

    return DataLoader(training_set, **training_params)

In [ ]:
for iter, (data, odata) in enumerate(zip(getLoader(), getOriginalLoader())):
    if iter == 3:
        break

    # print(iter)
    # print(data)
    img, annot = data['img'], data['annot']
    img = img
    img = img.permute(0, 2, 3, 1).numpy()
    rois = annot[:, :, :4].numpy()
    class_ids = annot[:, :, 4].numpy()

    oimg, oannot = odata['img'], odata['annot']
    oimg = oimg.permute(0, 2, 3, 1).numpy()
    oroiss = oannot[:, :, :4].numpy()
    oclass_ids = oannot[:, :, 4].numpy()

    print('img', img[0, 0, :3, :3])
    print('ori', oimg[0, 0, :3, :3])

    category_id_to_name = {v: k for v, k in enumerate(params.obj_list)}
    visualize(img[0], rois[0], class_ids[0], category_id_to_name, np.ones(len(rois[0])))
    visualize(oimg[0], oroiss[0], oclass_ids[0], category_id_to_name, np.ones(len(oroiss[0])))